<a href="https://colab.research.google.com/github/nanpolend/machine-learning/blob/master/RNA3D_deepseek_Gemini%E4%BF%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
RNA 3D結構預測比賽代碼
模組化設計，包含輸入處理、動態規劃核心算法、結構回溯、輸出格式轉換
"""

# --------------------------
# 模組1: 輸入處理與驗證
# --------------------------
def load_rna_sequence(file_path):
    """
    從FASTA文件讀取RNA序列
    參數:
        file_path (str): 輸入文件路徑
    返回:
        str: 大寫RNA序列(去除換行符)
    """
    try:
        with open(file_path, 'r') as f:
            # 跳過描述行
            sequence = ''.join(line.strip() for line in f.readlines()[1:])
            return sequence.upper()
    except Exception as e:
        print(f"檔案讀取錯誤: {e}")
        return None

def validate_sequence(seq):
    """
    驗證RNA序列有效性
    參數:
        seq (str): RNA序列
    返回:
        bool: 是否有效
    """
    valid_chars = {'A', 'U', 'C', 'G'}
    return all(c in valid_chars for c in seq)

# --------------------------
# 模組2: 動態規劃核心算法
# --------------------------
def initialize_dp_matrix(length):
    """
    初始化動態規劃矩陣
    參數:
        length (int): RNA序列長度
    返回:
        list: 二維矩陣(全零初始化)
    """
    return [[0] * length for _ in range(length)]

def nussinov_algorithm(sequence):
    """
    Nussinov動態規劃算法實現
    參數:
        sequence (str): RNA序列
    返回:
        list: 填充完成的DP矩陣
    """
    n = len(sequence)
    dp = initialize_dp_matrix(n)

    # 動態規劃填充(從短到長區間)
    for span in range(1, n):
        for i in range(n - span):
            j = i + span

            # 情況1: i與j配對
            if is_complementary(sequence[i], sequence[j]):
                dp[i][j] = dp[i+1][j-1] + 1

            # 情況2: 分割點k找最大值
            max_split = max([dp[i][k] + dp[k+1][j] for k in range(i, j)], default=0)

            # 取最大值
            dp[i][j] = max(dp[i][j], max_split)

    return dp

# --------------------------
# 模組3: 鹼基配對驗證
# --------------------------
def is_complementary(a, b):
    """
    檢查鹼基配對有效性
    參數:
        a (str): 第一個鹼基
        b (str): 第二個鹼基
    返回:
        bool: 是否有效配對
    """
    pairs = {('A', 'U'), ('U', 'A'),
             ('G', 'C'), ('C', 'G'),
             ('G', 'U'), ('U', 'G')}
    return (a, b) in pairs

# --------------------------
# 模組4: 結構回溯
# --------------------------
def traceback(dp, sequence):
    """
    回溯獲取最佳配對結構
    參數:
        dp (list): 填充完成的DP矩陣
        sequence (str): RNA序列
    返回:
        list: 配對位置tuple列表
    """
    n = len(sequence)
    pairs = []
    stack = [(0, n-1)]

    while stack:
        i, j = stack.pop()
        if i >= j:
            continue

        # 情況1: 當前位置配對
        if dp[i][j] == dp[i+1][j-1] + 1 and is_complementary(sequence[i], sequence[j]):
            pairs.append((i, j))
            stack.append((i+1, j-1))
        else:
            # 尋找分割點
            for k in range(i, j):
                if dp[i][j] == dp[i][k] + dp[k+1][j]:
                    stack.append((k+1, j))
                    stack.append((i, k))
                    break

    return pairs

# --------------------------
# 模組5: 輸出格式轉換
# --------------------------
def format_ct(sequence, pairs, filename):
    """
    生成CT格式文件
    參數:
        sequence (str): RNA序列
        pairs (list): 配對列表
        filename (str): 輸出文件名
    """
    pair_dict = {i:j for i,j in pairs}
    with open(filename, 'w') as f:
        f.write(f"{len(sequence)}\n")
        for idx, base in enumerate(sequence):
            pair = pair_dict.get(idx, -1)
            # CT格式: 序號 鹼基 前一位 後一位 配對位 序號(從1開始)
            f.write(f"{idx+1}\t{base}\t{idx}\t{idx+2}\t{pair+1 if pair != -1 else 0}\t{idx+1}\n")

# --------------------------
# 主流程控制
# --------------------------
def main(input_file, output_file):
    # 輸入處理
    seq = load_rna_sequence(input_file)
    if not seq or not validate_sequence(seq):
        print("無效輸入序列")
        return

    # 核心算法
    dp_matrix = nussinov_algorithm(seq)

    # 結構回溯
    predicted_pairs = traceback(dp_matrix, seq)

    # 輸出結果
    format_ct(seq, predicted_pairs, output_file)
    print(f"預測完成，結果已保存至 {output_file}")

# --------------------------
# 測試用例
# --------------------------
if __name__ == "__main__":
    # 測試用範例序列
    TEST_SEQ = "GUCGAC"

    # 單元測試
    assert is_complementary('A', 'U') == True
    assert is_complementary('G', 'U') == True
    assert is_complementary('A', 'C') == False

    # 整合測試
    test_pairs = traceback(nussinov_algorithm(TEST_SEQ), TEST_SEQ)
    print(f"測試序列配對結果: {test_pairs}")

    # 正式執行(替換實際文件路徑)
    # main("input.fasta", "output.ct")

測試序列配對結果: [(0, 5), (1, 4), (2, 3)]
